In [ ]:
import json
import pickle

import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

i = 0
book_url_list = []
while True:
    url = f'https://m.bqg90.com/json?sortid=0&page={i}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200 and response.text:
        response = json.loads(response.text)
        for item in response:
            book_url_list.append('https://m.bqg90.com' + item['url_list'])
        print(f'第{i}页爬取成功')
        i += 1
    else:
        print(f'在爬取第{i}页时出现错误, 错误码: {response.status_code}')
        break
book_url_list[:5], print(len(book_url_list))

book_url_list = list(set(book_url_list))

with open('book_url_list.pkl', 'wb') as f:
    pickle.dump(book_url_list, f)

In [ ]:
import pickle
book_url_list = list(set(book_url_list))
with open('book_url_list.pkl', 'wb') as f:
    pickle.dump(book_url_list, f)
with open('book_url_list.pkl', 'rb') as f:
    book_url_list = pickle.load(f)
book_url_list[:5], len(set(book_url_list)), len(book_url_list)

In [1]:
import requests
from scrapy import Selector
import tqdm
import json
import os
import pickle
from concurrent.futures import ThreadPoolExecutor, as_completed
from lxml import etree
import threading

# 创建锁
lock = threading.Lock()

def get_chapter(book_url, headers, pbar):
    response = requests.get(book_url, headers=headers)
    # response = etree.HTML(response.text)
    selector = Selector(text=response.text)
    # 获取书籍类型
    book_type = selector.xpath('//div[2]//dd[1]/span[2]/text()').extract_first()
    # 如果获取不到说明网页不存在
    if book_type is None:
        return None  
    book_type = book_type.split('：')[1]
    # 获取书籍名
    book_name = selector.xpath('//div[1]/div[2]/dl/dt/text()').extract_first()
    book_detail_url = book_url + 'list.html'
    response = requests.get(book_detail_url, headers=headers)
    selector = Selector(text=response.text)
    
    # 用于存放章节信息
    chapter_json_list = []
    # 添加章节信息
    i = 2
    while True:
        xpath_str = f'//dl/dd[{i}]/a/text()'
        chapter_name_text = selector.xpath(xpath_str).extract_first()
        if chapter_name_text:
            i += 1
            # 去掉不要的章节
            if chapter_name_text.startswith('第') and '章' in chapter_name_text:
                # 如果没有空格, 添加空格
                if ' ' not in chapter_name_text:
                    index = chapter_name_text.index('章')
                    chapter_name_text = chapter_name_text[:index + 1] + ' ' + chapter_name_text[index + 1:]
                # 添加章节名，并去除无用的信息
                # 去除章节名中的'第..章'
                chapter_name_text_clean = chapter_name_text.split(' ')[1]
                # # 去除章节名中的'【】'和'（）'和'()',可能会去除全部的章节名，但是我的训练集可以允许为空的章节名
                # chapter_name_text_clean = chapter_name_text_clean.split('【')[0].split('（')[0].split('(')[0]
                # chapter_name_text_clean = chapter_name_text_clean.split('】')[-1].split('）')[-1].split(')')[-1]
                        
                # 添加章节链接
                chapter_url_end = selector.xpath(
                    f'//dl/dd[{i}]/a/@href').extract_first()
                if chapter_url_end:
                    chapter_url = 'https://m.bqg90.com/' + chapter_url_end
                else:
                    continue
                    
                chapter_json_list.append({
                    'book_name': book_name,
                    'book_type': book_type,
                    'chapter_name': chapter_name_text_clean,
                    'chapter_index': int(chapter_name_text.split(' ')[0][1:-1]),
                    'chapter_url': chapter_url})
            else:
                break
        else:
            break
    
    with lock:        
        with open('get_chapter_progress.txt', 'w') as f:    
            f.write(book_url)
            
        with open('chapter_json.json', 'a', encoding='utf-8') as f:
            for chapter_json in chapter_json_list:
                f.write(json.dumps(chapter_json, ensure_ascii=False) + '\n')
        pbar.update(1)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

with open('book_url_list.pkl', 'rb') as f:
    book_url_list = pickle.load(f)

if os.path.exists('get_chapter_progress.txt'):
    with open('get_chapter_progress.txt', 'r') as f:
        start_url = f.read().strip()
else:
    start_url = None

if start_url:
    start_index = book_url_list.index(start_url) + 1
    book_url_list = book_url_list[start_index:10000]

with tqdm.tqdm(total=len(book_url_list)) as pbar:
    with ThreadPoolExecutor(max_workers=128) as executor:
        for book_url in book_url_list:
            executor.submit(get_chapter, book_url, headers, pbar)

 69%|██████▉   | 2587/3750 [24:18<10:55,  1.77it/s] 


In [1]:
import requests
from scrapy import Selector
import tqdm
import json
import os
from concurrent.futures import ThreadPoolExecutor
import threading

lock = threading.Lock()

def get_book_text(chapter_data, headers, pbar):
    response = requests.get(chapter_data['chapter_url'], headers=headers)
    selector = Selector(text=response.text)
    paragraph_list = selector.xpath('//*[@id="chaptercontent"]/text()').extract()
    if paragraph_list:
        text = ''
        for paragraph in paragraph_list:
            if paragraph.endswith('。'):
                text += paragraph + '\n'
        data_json = {
            'book_name': chapter_data['book_name'],
            'book_type': chapter_data['book_type'],
            'chapter_name': chapter_data['chapter_name'],
            'chapter_index': chapter_data['chapter_index'],
            'book_text': text.replace('\t', '').replace('\r', '')
        }
            
        with lock:
            with open('get_book_text_progress.txt', 'r') as f:
                index = int(f.read().strip())
            with open('get_book_text_progress.txt', 'w') as f:
                f.write(str(index + 1))
            with open('book_text.json', 'a', encoding='utf-8') as f:
                f.write(json.dumps(data_json, ensure_ascii=False) + '\n')
            pbar.update(1)

if os.path.exists('get_book_text_progress.txt'):
    with open('get_book_text_progress.txt', 'r') as f:
        start_index = int(f.read().strip())
else:
    with open('get_book_text_progress.txt', 'w') as f:
        f.write('0')
    start_index = 0
    
with open('chapter_json.json', 'r', encoding='utf-8') as f:
    chapter_data_list = [json.loads(line) for line in f]
    
chapter_data_list = chapter_data_list[start_index:300000]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

with tqdm.tqdm(total=len(chapter_data_list)) as pbar:
    with ThreadPoolExecutor(max_workers=32) as executor:
        for chapter_data in chapter_data_list:
            executor.submit(get_book_text, chapter_data, headers, pbar)

 63%|██████▎   | 181079/285392 [9:47:52<5:41:15,  5.09it/s]    